In [2]:
import cv2
import mediapipe as mp
import time
import math
import numpy as np

In [ ]:
class handDetector():
    def __init__(self, mode=False, maxHands=2, detectionCon=0.8, trackCon=0.8):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands,
                                        self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils
        self.tipIds = [4, 8, 12, 16, 20]

    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        # print(results.multi_hand_landmarks)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms,
                                               self.mpHands.HAND_CONNECTIONS)

        return img

    def findPosition(self, img, handNo=0, draw=True):
        xList = []
        yList = []
        bbox = []
        self.lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                # print(id, lm)
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                xList.append(cx)
                yList.append(cy)
                # print(id, cx, cy)
                self.lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

            xmin, xmax = min(xList), max(xList)
            ymin, ymax = min(yList), max(yList)
            bbox = xmin, ymin, xmax, ymax

            if draw:
                cv2.rectangle(img, (xmin - 20, ymin - 20), (xmax + 20, ymax + 20),
                              (0, 255, 0), 2)

        return self.lmList, bbox

    def fingersUp(self):
        fingers = []
        # Thumb
        if self.lmList[self.tipIds[0]][1] > self.lmList[self.tipIds[0] - 1][1]:
            fingers.append(1)
        else:
            fingers.append(0)

        # Fingers
        for id in range(1, 5):

            if self.lmList[self.tipIds[id]][2] < self.lmList[self.tipIds[id] - 2][2]:
                fingers.append(1)
            else:
                fingers.append(0)

        # totalFingers = fingers.count(1)

        return fingers

    def findDistance(self, p1, p2, img, draw=True,r=15, t=3):
        x1, y1 = self.lmList[p1][1:]
        x2, y2 = self.lmList[p2][1:]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        if draw:
            cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), t)
            cv2.circle(img, (x1, y1), r, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), r, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (cx, cy), r, (0, 0, 255), cv2.FILLED)
        length = math.hypot(x2 - x1, y2 - y1)

        return length, img, [x1, y1, x2, y2, cx, cy]


def main():
    pTime = 0
    cTime = 0
    cap = cv2.VideoCapture(0)
    detector = handDetector()
    while True:
        success, img = cap.read()
        img = detector.findHands(img)
        lmList, bbox = detector.findPosition(img)
        if len(lmList) != 0:
            print(lmList[4])

        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime

        cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3,
                    (255, 0, 255), 3)

        cv2.imshow("Image", img)
        cv2.waitKey(1)


if __name__ == "__main__":
    main()

[4, 565, 289]
[4, 565, 289]
[4, 530, 231]
[4, 530, 232]
[4, 527, 232]
[4, 529, 227]
[4, 527, 237]
[4, 535, 252]
[4, 415, 212]
[4, 407, 240]
[4, 464, 197]
[4, 469, 196]
[4, 472, 201]
[4, 469, 205]
[4, 421, 219]
[4, 424, 223]
[4, 399, 215]
[4, 395, 204]
[4, 390, 211]
[4, 391, 206]
[4, 388, 205]
[4, 389, 205]
[4, 386, 198]
[4, 385, 199]
[4, 381, 191]
[4, 381, 194]
[4, 374, 194]
[4, 379, 199]
[4, 361, 187]
[4, 371, 194]
[4, 366, 197]
[4, 362, 196]
[4, 356, 193]
[4, 359, 193]
[4, 355, 184]
[4, 351, 187]
[4, 360, 188]
[4, 356, 186]
[4, 349, 187]
[4, 359, 188]
[4, 350, 190]
[4, 352, 192]
[4, 359, 192]
[4, 358, 183]
[4, 356, 187]
[4, 346, 186]
[4, 355, 187]
[4, 351, 188]
[4, 351, 185]
[4, 351, 194]
[4, 355, 193]
[4, 357, 193]
[4, 359, 198]
[4, 358, 196]
[4, 362, 206]
[4, 365, 209]
[4, 383, 210]
[4, 378, 214]
[4, 402, 206]
[4, 397, 220]
[4, 410, 215]
[4, 413, 223]
[4, 407, 214]
[4, 405, 218]
[4, 403, 220]
[4, 408, 222]
[4, 423, 249]
[4, 422, 241]
[4, 447, 259]
[4, 451, 261]
[4, 463, 255]
[4, 46

[4, 401, 193]
[4, 399, 197]
[4, 398, 204]
[4, 397, 202]
[4, 396, 205]
[4, 397, 205]
[4, 398, 206]
[4, 399, 205]
[4, 401, 206]
[4, 402, 206]
[4, 399, 209]
[4, 398, 208]
[4, 395, 211]
[4, 394, 212]
[4, 391, 213]
[4, 390, 213]
[4, 385, 213]
[4, 384, 215]
[4, 382, 211]
[4, 381, 211]
[4, 378, 214]
[4, 376, 214]
[4, 377, 215]
[4, 384, 219]
[4, 384, 220]
[4, 390, 221]
[4, 390, 226]
[4, 385, 231]
[4, 384, 231]
[4, 374, 239]
[4, 372, 238]
[4, 366, 238]
[4, 363, 238]
[4, 361, 232]
[4, 361, 231]
[4, 364, 225]
[4, 363, 225]
[4, 368, 223]
[4, 367, 225]
[4, 370, 223]
[4, 371, 225]
[4, 379, 221]
[4, 377, 223]
[4, 383, 218]
[4, 382, 218]
[4, 380, 221]
[4, 378, 220]
[4, 375, 221]
[4, 375, 221]
[4, 374, 221]
[4, 372, 221]
[4, 372, 221]
[4, 372, 221]
[4, 371, 225]
[4, 371, 225]
[4, 372, 224]
[4, 376, 233]
[4, 378, 235]
[4, 394, 243]
[4, 391, 246]
[4, 400, 236]
[4, 399, 237]
[4, 401, 228]
[4, 401, 229]
[4, 404, 224]
[4, 402, 224]
[4, 409, 226]
[4, 409, 228]
[4, 415, 230]
[4, 416, 231]
[4, 420, 234]
[4, 42

[4, 519, 224]
[4, 518, 222]
[4, 520, 225]
[4, 520, 222]
[4, 523, 227]
[4, 524, 226]
[4, 524, 225]
[4, 524, 225]
[4, 525, 225]
[4, 526, 224]
[4, 529, 226]
[4, 528, 224]
[4, 528, 223]
[4, 529, 224]
[4, 530, 220]
[4, 531, 221]
[4, 531, 222]
[4, 532, 220]
[4, 533, 222]
[4, 532, 222]
[4, 533, 222]
[4, 531, 222]
[4, 534, 222]
[4, 534, 220]
[4, 535, 221]
[4, 534, 221]
[4, 536, 225]
[4, 536, 222]
[4, 537, 223]
[4, 537, 223]
[4, 536, 225]
[4, 536, 225]
[4, 536, 222]
[4, 538, 224]
[4, 535, 222]
[4, 537, 223]
[4, 533, 222]
[4, 534, 224]
[4, 512, 215]
[4, 509, 221]
[4, 508, 222]
[4, 507, 228]
[4, 509, 228]
[4, 512, 231]
[4, 518, 231]
[4, 521, 231]
[4, 522, 229]
[4, 522, 230]
[4, 518, 222]
[4, 521, 225]
[4, 524, 230]
[4, 522, 225]
[4, 523, 230]
[4, 525, 231]
[4, 525, 234]
[4, 525, 233]
[4, 523, 231]
[4, 522, 231]
[4, 522, 231]
[4, 520, 229]
[4, 521, 233]
[4, 516, 228]
[4, 517, 229]
[4, 517, 230]
[4, 517, 230]
[4, 517, 229]
[4, 520, 229]
[4, 518, 225]
[4, 519, 226]
[4, 519, 225]
[4, 520, 228]
[4, 52

[4, 473, 324]
[4, 476, 313]
[4, 475, 315]
[4, 530, 236]
[4, 441, 360]
[4, 443, 358]
[4, 452, 349]
[4, 452, 351]
[4, 462, 349]
[4, 462, 348]
[4, 471, 347]
[4, 470, 346]
[4, 474, 347]
[4, 473, 347]
[4, 476, 346]
[4, 474, 347]
[4, 474, 344]
[4, 475, 344]
[4, 476, 341]
[4, 476, 341]
[4, 477, 339]
[4, 477, 339]
[4, 476, 339]
[4, 476, 339]
[4, 476, 338]
[4, 476, 338]
[4, 475, 338]
[4, 475, 338]
[4, 473, 338]
[4, 472, 339]
[4, 473, 338]
[4, 473, 338]
[4, 473, 336]
[4, 474, 336]
[4, 474, 337]
[4, 474, 337]
[4, 474, 337]
[4, 475, 336]
[4, 475, 336]
[4, 476, 336]
[4, 475, 338]
[4, 475, 338]
[4, 475, 337]
[4, 475, 337]
[4, 475, 337]
[4, 475, 336]
[4, 474, 339]
[4, 474, 339]
[4, 473, 338]
[4, 474, 339]
[4, 473, 339]
[4, 473, 338]
[4, 473, 338]
[4, 473, 338]
[4, 472, 337]
[4, 472, 337]
[4, 471, 338]
[4, 471, 339]
[4, 471, 338]
[4, 471, 336]
[4, 468, 339]
[4, 467, 339]
[4, 468, 338]
[4, 468, 339]
[4, 469, 341]
[4, 469, 340]
[4, 468, 340]
[4, 469, 339]
[4, 468, 342]
[4, 465, 342]
[4, 462, 340]
[4, 46

[4, 474, 192]
[4, 465, 200]
[4, 458, 191]
[4, 452, 187]
[4, 381, 189]
[4, 379, 187]
[4, 401, 209]
[4, 407, 213]
[4, 460, 208]
[4, 471, 199]
[4, 438, 199]
[4, 439, 192]
[4, 382, 183]
[4, 385, 186]
[4, 394, 193]
[4, 386, 197]
[4, 475, 202]
[4, 472, 206]
[4, 452, 218]
[4, 454, 212]
[4, 379, 177]
[4, 380, 176]
[4, 385, 192]
[4, 388, 199]
[4, 458, 186]
[4, 454, 198]
[4, 454, 196]
[4, 454, 200]
[4, 384, 165]
[4, 380, 174]
[4, 396, 186]
[4, 392, 189]
[4, 453, 196]
[4, 458, 186]
[4, 451, 178]
[4, 452, 176]
[4, 418, 183]
[4, 385, 177]
[4, 366, 198]
[4, 365, 195]
[4, 471, 361]
[4, 472, 370]
[4, 472, 370]
[4, 478, 352]
[4, 468, 341]
[4, 462, 374]
[4, 459, 376]
[4, 569, 282]
[4, 568, 279]
[4, 563, 284]
[4, 562, 232]
[4, 561, 243]
[4, 525, 262]
[4, 454, 296]
[4, 469, 344]
[4, 462, 350]
[4, 459, 350]
[4, 457, 362]
[4, 455, 372]
[4, 565, 292]
[4, 541, 276]
[4, 542, 276]
[4, 548, 257]
[4, 553, 281]
[4, 561, 285]
[4, 569, 290]
[4, 570, 296]
[4, 570, 302]
[4, 573, 296]
[4, 555, 265]
[4, 548, 255]
[4, 55